In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px

# Title
st.title("🏠 House Price Dashboard")

# Load Excel file
@st.cache_data
def load_data():
    df = pd.read_excel("House_Prices.xlsx", sheet_name="Sheet1")
    key_columns = ['MSZoning', 'LotShape', 'Street', 'Neighborhood', 'HouseAge', 'TotalSF', 'TotalBaths', 'SalePrice']
    return df[key_columns]

df = load_data()

# Sidebar Filters
st.sidebar.header("🔎 Filter Criteria")

mszoning = st.sidebar.multiselect("MS Zoning", options=df['MSZoning'].unique(), default=df['MSZoning'].unique())
lotshape = st.sidebar.multiselect("Lot Shape", options=df['LotShape'].unique(), default=df['LotShape'].unique())
street = st.sidebar.multiselect("Street Type", options=df['Street'].unique(), default=df['Street'].unique())
neighborhood = st.sidebar.multiselect("Neighborhood", options=df['Neighborhood'].unique(), default=df['Neighborhood'].unique())

houseage_min, houseage_max = st.sidebar.slider(
    "House Age Range", int(df['HouseAge'].min()), int(df['HouseAge'].max()),
    (int(df['HouseAge'].min()), int(df['HouseAge'].max()))
)

totalsf_min, totalsf_max = st.sidebar.slider(
    "Total SF Range", int(df['TotalSF'].min()), int(df['TotalSF'].max()),
    (int(df['TotalSF'].min()), int(df['TotalSF'].max()))
)

saleprice_min, saleprice_max = st.sidebar.slider(
    "Sale Price Range", int(df['SalePrice'].min()), int(df['SalePrice'].max()),
    (int(df['SalePrice'].min()), int(df['SalePrice'].max()))
)

totalbaths = st.sidebar.multiselect("Total Baths", sorted(df['TotalBaths'].unique()), default=sorted(df['TotalBaths'].unique()))

# Apply filters
filtered_df = df[
    (df['MSZoning'].isin(mszoning)) &
    (df['LotShape'].isin(lotshape)) &
    (df['Street'].isin(street)) &
    (df['Neighborhood'].isin(neighborhood)) &
    (df['HouseAge'] >= houseage_min) & (df['HouseAge'] <= houseage_max) &
    (df['TotalSF'] >= totalsf_min) & (df['TotalSF'] <= totalsf_max) &
    (df['SalePrice'] >= saleprice_min) & (df['SalePrice'] <= saleprice_max) &
    (df['TotalBaths'].isin(totalbaths))
]

# Display Filtered Data
st.subheader("📊 Filtered House Data")
st.write(f"Showing {len(filtered_df)} results")
st.dataframe(filtered_df)

# Keywords
st.write("MSZoning -> RL: Residential Low Density, RM: Residential Medium Density, C: Commercial, FV: Floating Village Residential, RH: Residential High Density")
st.write("LotShape -> Reg: Regular, IR1: Slightly irregular, IR2: Moderately Irregular, IR3: Irregular")

# Summary Stats
if not filtered_df.empty:
    st.subheader("📈 Sale Price Summary")
    st.write(filtered_df['SalePrice'].describe())

    # Histogram of Sale Prices
    st.subheader("📉 Distribution of Sale Prices")
    fig_hist = px.histogram(filtered_df, x="SalePrice", nbins=30, title="Sale Price Distribution")
    st.plotly_chart(fig_hist)

    # Boxplot of Sale Prices by Neighborhood
    st.subheader("📦 Sale Price by Neighborhood")
    fig_box = px.box(filtered_df, x="Neighborhood", y="SalePrice", color = "Neighborhood", title="Sale Price by Neighborhood")
    st.plotly_chart(fig_box)

    # Scatter Plot of TotalSF vs SalePrice
    st.subheader("📍 Total Square Feet vs Sale Price")
    fig_scatter = px.scatter(filtered_df, x="TotalSF", y="SalePrice", color="Neighborhood",
                             title="TotalSF vs SalePrice by Neighborhood", hover_data=["HouseAge"])
    st.plotly_chart(fig_scatter)
else:
    st.warning("No data found for selected filters.")